In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/housingdata/housing.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# Keep only records that have ocean_proximity
# For multiple conditions, each condition should be in Parentheses
df = df[(df['ocean_proximity'] == 'INLAND') | (df['ocean_proximity'] == '<1H OCEAN')]
df

In [ ]:
df_use = df.copy()
del df_use['ocean_proximity']

In [ ]:
df_use.isnull().sum()
# total_bedrooms has 157 null values

In [ ]:
# median value for population
np.median(df_use['population'])
# 1195

In [ ]:
# Plot median_house_value
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
sns.histplot(df_use.median_house_value, bins=50)

In [ ]:
median_price_logs = np.log1p(df_use.median_house_value)
median_price_logs

In [ ]:
sns.histplot(median_price_logs)
# no long-tail distrubution, we get normal distrubution

In [ ]:
# Prepare and shuffle the df_use
n = len(df_use)
n_val = int(n *.2)
n_test = int(n * .2)
n_train = n - n_val - n_test 
print(n, n_train, n_val, n_test)

In [ ]:
# create a range of size n
idx = np.arange(n)
idx

In [ ]:
# Now we shuffle
# set the seed
np.random.seed(42)

In [ ]:
np.random.shuffle(idx)
idx

In [ ]:
df_train = df_use.iloc[idx[:n_train]]
df_val = df_use.iloc[idx[n_train:n_val+n_train]]
df_test = df_use.iloc[idx[n_train+n_val:]]
len(df_train), len(df_val), len(df_test)

In [ ]:
# mean value of total_bedrooms
total_bedrooms_mean = df_train['total_bedrooms'].mean()
total_bedrooms_mean
# 542.553

In [ ]:
# the index values are all random in the new dataset. 
# we can use reset_index to reset them
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# these are all the feature Matrix X


In [ ]:
# Now we define our Y - median_home_value
y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)


In [ ]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [ ]:
df_train.iloc[2]

In [ ]:
df_train.shape, df_train.isnull().sum()

In [ ]:
base = list(df_train.columns.values)
base

In [ ]:
df_train_mean = df_train.copy()
df_train_mean = df_train_mean[base].fillna(542.552956325786)
X_train_mean = df_train_mean.values
X_train_mean.shape, y_train.shape

In [ ]:
df_train_zero = df_train.copy()
df_train_zero = df_train_zero[base].fillna(0)
X_train_zero = df_train_zero.values

In [ ]:
def train_linear_regression(X, y):
#     add in bias term
    ones = np.ones(X.shape[0])
#     stack the columns to create new feature matrix
    X = np.column_stack([ones, X])
    
#     normal equation
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [ ]:
mean_w0, mean_w = train_linear_regression(X_train_mean, y_train)
mean_w0, mean_w

In [ ]:
zero_w0, zero_w = train_linear_regression(X_train_zero, y_train)
zero_w0, zero_w

In [ ]:
y_pred_mean = mean_w0 + X_train_mean.dot(mean_w)
y_pred_mean

In [ ]:
y_pred_zero = zero_w0 + X_train_zero.dot(zero_w)
y_pred_zero

In [ ]:
# Plot the predictions
sns.histplot(y_pred_mean, color="red", alpha=0.5, bins=50)
sns.histplot(y_train, color="green", alpha=0.5, bins=50)

In [ ]:
# Plot the predictions
sns.histplot(y_pred_zero, color="red", alpha=0.5, bins=50)
sns.histplot(y_train, color="green", alpha=0.5, bins=50)

In [ ]:
def rmse(y, y_pred):
    s_error =  (y - y_pred) ** 2
    mean_s_error = s_error.mean()
    return np.sqrt(mean_s_error)

In [ ]:
# mean total_bedrooms
rmse(y_train, y_pred_mean)

In [ ]:
# 0 total bedrooms
rmse(y_train, y_pred_zero)

In [ ]:
def prepare_X(df, fill):
    df = df.copy()
    features = base
    df_num = df[features].fillna(fill)
    X = df_num.values

    return X
    

In [ ]:
X_train_mean = prepare_X(df_train, total_bedrooms_mean)
X_train_mean

In [ ]:
#  Train DF - df_train - set total_bedrooms to mean value
X_train_mean = prepare_X(df_train, total_bedrooms_mean)
mean_w0, mean_w = train_linear_regression(X_train_mean, y_train)
# prepare the df_val - we prepare it the same way. 
X_val_mean = prepare_X(df_val, total_bedrooms_mean)
y_pred = mean_w0 + X_val_mean.dot(mean_w)
mean_score = round(rmse(y_val, y_pred), 2)
mean_score

In [ ]:
#  Train DF - df_train - set total_bedrooms to 0
X_train_zero = prepare_X(df_train, 0)
zero_w0, zero_w = train_linear_regression(X_train_zero, y_train)
# prepare the df_val - we prepare it the same way. 
X_val_zero = prepare_X(df_val, 0)
y_pred = zero_w0 + X_val_zero.dot(zero_w)
zero_score = round(rmse(y_val, y_pred), 2)
zero_score
# Both are equally good. 